In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import os
import wget
from sklearn.metrics import accuracy_score
from utils import *

## Overview
We have a database (ULL_database) with information about primary and secondary education students in the Canary Islands 
for 4 academic years. There is information about their academic performance and 
contextual information (about their families, teachers, and school). The database contains a subset of data 
in the form of panel data, meaning information about the same students at different points in time (ULL_panel_data).

Machine learning algorithms can be used to predict at-risk students. 
A student is considered at risk if they are anticipated to have low academic performance in the future. 
Detecting these students would allow for corrective measures to be taken in advance.

As a measure of academic performance, we have the variables "scores".
We have academic performance in Mathematics and in Spanish Language

We specify a model to predict at-risk students. Utilizing the panel data,
the model aims to forecast whether the student will be at risk in the future (in 6th grade)
based on various predictors of current academic performance (3rd grade).

Each observation (row) in ULL_panel_data is a student, with their academic performance in sixth grade 
and their predictors of academic performance from third grade (columns).

## Load and preprocess data

In [2]:
DATA = 'data/'
data = pd.read_csv(os.path.join(DATA, 'ULL_panel_data.csv'), sep=';')

In [3]:
data

,id_grade,id_student_16_19,score_MAT,level_MAT,score_LEN,level_LEN,id_student,id_student_original,id_year,id_class_group,...,p331a,p331b,p331c,p331d,p331e,p331f,p331g,p331j,pfc,rep
0,6,1,474.9944,2,385.1411,1,20342,1431,2016,A,...,4.0,4.0,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
1,6,2,508.8362,3,469.4856,2,2819,1432,2016,A,...,4.0,NaN,4.0,4.0,3.0,4.0,NaN,NaN,NaN,NaN
2,6,3,590.2816,3,591.1398,3,19276,1436,2016,A,...,4.0,4.0,4.0,4.0,2.0,4.0,NaN,NaN,NaN,NaN
3,6,5,394.4247,1,493.7984,2,14078,1439,2016,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,6,530.0070,3,500.0860,3,1695,1447,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15969,6,17267,599.0310,3,615.6177,4,12265,40236,2016,A,...,4.0,NaN,4.0,4.0,4.0,3.0,NaN,NaN,NaN,NaN
15970,6,17268,538.5835,3,647.9100,4,15982,40238,2016,A,...,4.0,NaN,4.0,4.0,4.0,4.0,NaN,NaN,NaN,NaN
15971,6,17269,537.8327,3,445.1313,2,9965,40240,2016,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15972,6,17270,468.8731,2,546.8035,3,1137,40246,2016,B,...,3.0,4.0,3.0,3.0,3.0,3.0,NaN,NaN,NaN,NaN


In [4]:
# Select only the data we want to work for
data = data[['id_student_16_19', 'score_MAT', 'score_LEN', 'score_MAT3', 'score_LEN3', 'a1',
             'mother_education', 'father_education', 'mother_occupation', 'father_occupation', 
             'inmigrant_second_gen', 'start_schooling_age', 'books', 'f12a', 'public_private', 
             'capital_island', 'd14', 'ESCS', 'id_school']]

In [5]:
# Drop observations with missing data in any of the variables that we will use in the models
# Here, synthetic data methods can be used instead to fill in missing values

missing_columns = ['score_MAT3', 'a1', 'mother_education', 'father_education',
    'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
    'start_schooling_age', 'books', 'f12a', 'public_private',
    'capital_island', 'd14']

data = data.dropna(subset=missing_columns)

In [6]:
data

,id_student_16_19,score_MAT,score_LEN,score_MAT3,score_LEN3,a1,mother_education,father_education,mother_occupation,father_occupation,inmigrant_second_gen,start_schooling_age,books,f12a,public_private,capital_island,d14,ESCS,id_school
4,6,530.0070,500.0860,368.65,339.47,1,3.0,1.0,4.0,2.0,1.0,1.0,1.0,1.0,2,2,4.0,0.132756,2443
5,7,531.9280,459.4065,387.36,566.44,1,4.0,4.0,4.0,3.0,1.0,1.0,4.0,5.0,2,1,1.0,1.069410,1368
7,9,578.3741,630.4484,549.89,635.53,1,2.0,2.0,3.0,2.0,1.0,2.0,2.0,2.0,2,1,1.0,-1.166950,2500
8,10,481.1748,497.1981,592.00,668.26,2,4.0,4.0,4.0,4.0,1.0,1.0,3.0,4.0,1,1,1.0,0.976453,1610
11,13,521.5593,655.0537,490.28,524.98,2,4.0,4.0,3.0,3.0,1.0,1.0,2.0,5.0,2,1,1.0,-0.134441,1859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15968,17266,522.6458,611.0034,574.83,591.86,1,4.0,3.0,3.0,3.0,1.0,1.0,4.0,2.0,2,1,2.0,0.759928,2413
15969,17267,599.0310,615.6177,629.84,460.75,1,4.0,3.0,4.0,3.0,1.0,1.0,4.0,2.0,2,1,2.0,1.410322,2203
15970,17268,538.5835,647.9100,600.20,542.78,1,4.0,2.0,3.0,3.0,1.0,1.0,2.0,3.0,2,1,2.0,0.035227,2095
15971,17269,537.8327,445.1313,610.26,600.15,2,4.0,1.0,3.0,3.0,1.0,1.0,3.0,2.0,2,1,2.0,0.382896,2097


In [7]:
cols = data.columns 
data = pd.DataFrame(data.values.flatten().reshape(-1, data.shape[1]), columns=cols)

In [8]:
# Generate quartiles of scores in sixth grade
data['score_MATq'] = pd.qcut(data['score_MAT'], 4, labels=["1", "2", "3","4"])
data['score_MATq'] = data['score_MATq'].astype(int)
data['score_LENq'] = pd.qcut(data['score_LEN'], 4, labels=["1", "2", "3","4"])
data['score_LENq'] = data['score_LENq'].astype(int)

In [9]:
# Generate median and percentiles 25 and 75 of socioeconomic status (ESCS)
median_ESCS = data['ESCS'].median()
p25_ESCS = data['ESCS'].quantile(0.25)
p75_ESCS = data['ESCS'].quantile(0.75)

# Initialize with null values
data['ESCS_median'] = pd.Series([np.nan] * len(data))
data.loc[data['ESCS'] >= median_ESCS, 'ESCS_median'] = 2
data.loc[data['ESCS'] < median_ESCS, 'ESCS_median'] = 1
data.loc[data['ESCS_median'] == 0, 'ESCS_median'] = np.nan

# Initialize with null values
data['ESCS_p25_p75'] = pd.Series([np.nan] * len(data))
data.loc[data['ESCS'] >= p75_ESCS, 'ESCS_p25_p75'] = 2
data.loc[data['ESCS'] < p25_ESCS, 'ESCS_p25_p75'] = 1
data.loc[(data['ESCS'] >= p25_ESCS) & (data['ESCS'] < p75_ESCS), 'ESCS_p25_p75'] = np.nan

# Some data corrections to make the final results
# Variable d14 top category(4) is the "bad" category (more than 50% of teachers change school), so the results must be inverted
data['d14'] = data['d14'].apply(lambda x: 1 if x == 1 else 0)

## Models

The goal of the model is to predict the academic performance in sixth grade ($Y_t$)
using information from the same student in third grade, specifically:

1.  Academic performance in third grade ($Y_{t-1}$)

2.  Sensitive factors or circumstances ($C$)

3.  Predictors uncorrelated with circumstances, also called "effort" ($X$)

**Model 1**:    $$Y_t = α + β1Y_{t-1} + ε$$

**Model 2**:    $$Y_t = α + β1Y_{t-1} + β2C + ε$$

**Model 3**:    

> First step: $$Y_{t-1} = α + β2C + ν$$

- Recover the prediction of $Y_{t-1}$ (academic performance due to circumstances, $C$): $\hat{Y}_{t-1}$

- Recover the residual $ν$ (academic performance due to effort, $X$): $\hat{ν}$

> Second step: $$Y_t = α + β1\hat{Y}_{t-1} + β2\hat{ν} + ε$$

- Recover the prediction of $Y_t$ only due to $\hat{Y}_{t-1}$ (only due to circumstances)

- Recover the prediction of $Y_t$ only due to $\hat{ν}$ (only due to effort)

In theory...

**Model 1**: Using only the academic performance in third grade (benchmark)

**Model 2**: Using the academic performance + circumstances in third grade (less fair - more socially desirable)

**Model 3**: Using the circumstances + effort in third grade (close to Model 2)

- Prediction exclusively of circumstances of Model 3 (much less fair - much more socially desirable)
    
- Prediction exclusively of effort of Model 3 (much more fair - much less socially desirable)

Let's prove it

In [10]:
# Variables for the models
Y_t_1 = "score_MAT3"
data = data.sample(frac=1, random_state=42).reset_index(drop=True)
split = int(len(data) * 0.7)

train_data = data.iloc[:split]
test_data = data.iloc[split:]

train_c = train_data[["a1", "mother_education", "father_education", "mother_occupation", "father_occupation", 
      "inmigrant_second_gen", "start_schooling_age", "books", "f12a", "public_private", "capital_island", "d14"]]
test_c = test_data[["a1", "mother_education", "father_education", "mother_occupation", "father_occupation", 
      "inmigrant_second_gen", "start_schooling_age", "books", "f12a", "public_private", "capital_island", "d14"]]
circumstances = ["a1", "mother_education", "father_education", "mother_occupation", "father_occupation", 
      "inmigrant_second_gen", "start_schooling_age", "books", "f12a", "public_private", "capital_island", "d14"]

# Dummy variables (all variables C are categorical variables)
train_dummy_variables = pd.get_dummies(train_c, columns=circumstances, drop_first = True)
test_dummy_variables = pd.get_dummies(test_c, columns=circumstances, drop_first = True)

# Join Y_t_1 + C
train_data_combined = pd.concat([train_data[Y_t_1], train_dummy_variables], axis=1)
test_data_combined = pd.concat([test_data[Y_t_1], test_dummy_variables], axis=1)

In [11]:
test_data

,id_student_16_19,score_MAT,score_LEN,score_MAT3,score_LEN3,a1,mother_education,father_education,mother_occupation,father_occupation,...,f12a,public_private,capital_island,d14,ESCS,id_school,score_MATq,score_LENq,ESCS_median,ESCS_p25_p75
5803,5202.0,498.8352,559.8265,482.84,495.24,1.0,1.0,1.0,4.0,3.0,...,2.0,1.0,1.0,1,-0.541568,2128.0,2,3,1.0,1.0
5804,15188.0,625.0212,549.8165,592.76,612.76,1.0,4.0,4.0,3.0,4.0,...,5.0,1.0,1.0,1,1.599505,2235.0,4,3,2.0,2.0
5805,5975.0,518.0978,571.5323,458.78,598.39,2.0,4.0,4.0,4.0,4.0,...,5.0,1.0,1.0,1,0.976453,1426.0,3,3,2.0,2.0
5806,8069.0,508.2131,444.0736,444.45,618.62,2.0,3.0,3.0,3.0,3.0,...,3.0,2.0,1.0,0,0.008032,2168.0,3,1,1.0,NaN
5807,2116.0,718.4857,532.0629,666.70,531.61,1.0,4.0,4.0,3.0,3.0,...,4.0,2.0,1.0,0,0.288806,2409.0,4,3,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8285,11919.0,630.6860,744.5259,554.04,540.68,2.0,4.0,4.0,4.0,4.0,...,5.0,1.0,1.0,1,1.947430,1608.0,4,4,2.0,2.0
8286,10754.0,431.4843,616.2399,649.20,530.06,2.0,3.0,4.0,4.0,4.0,...,5.0,1.0,1.0,1,0.536982,1295.0,1,4,2.0,NaN
8287,11180.0,631.2706,633.2511,494.00,583.24,2.0,3.0,4.0,3.0,3.0,...,4.0,2.0,1.0,1,1.502406,1983.0,4,4,2.0,2.0
8288,1766.0,623.5959,414.1856,415.69,502.65,1.0,4.0,3.0,3.0,3.0,...,2.0,1.0,1.0,1,0.289729,2504.0,4,1,2.0,NaN


In [12]:
test_data.shape

(2487, 23)

In [13]:
# Model 1
model1 = sm.OLS(train_data["score_MAT"], sm.add_constant(train_data[Y_t_1])).fit()
model1.summary()
test_data.loc[:, 'model1_pred'] = model1.predict(sm.add_constant(test_data[Y_t_1]))

/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/99475333.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'model1_pred'] = model1.predict(sm.add_constant(test_data[Y_t_1]))


In [14]:
# Model 1
# model1 = sm.OLS(data["score_MAT"], sm.add_constant(data[Y_t_1])).fit()
# print(model1.summary())
# data['model1_pred'] = model1.fittedvalues

In [15]:
# Model 2
# model2 = sm.OLS(data["score_MAT"], sm.add_constant(data_combined.astype(np.float64))).fit()
# print(model2.summary())
# data['model2_pred'] = model2.fittedvalues

In [16]:
# Model 2
model2 = sm.OLS(train_data["score_MAT"], sm.add_constant(train_data_combined.astype(np.float64))).fit()
print(model2.summary())
test_data.loc[:, 'model2_pred'] = model2.predict(sm.add_constant(test_data_combined.astype(np.float64)))

                            OLS Regression Results                            
Dep. Variable:              score_MAT   R-squared:                       0.271
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     79.37
Date:                Wed, 18 Sep 2024   Prob (F-statistic):               0.00
Time:                        15:06:37   Log-Likelihood:                -34035.
No. Observations:                5803   AIC:                         6.813e+04
Df Residuals:                    5775   BIC:                         6.831e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/3183405957.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.loc[:, 'model2_pred'] = model2.predict(sm.add_constant(test_data_combined.astype(np.float64)))


In [17]:
# Model 3
# model3 = sm.OLS(data["score_MAT3"], sm.add_constant(dummy_variables.astype(np.float64))).fit()
# print(model3.summary())

# First step
# data['Y_t_1_hat'] = model3.fittedvalues
# data['ν_hat'] = model3.resid

# Second step
# model4 = sm.OLS(data["score_MAT"], sm.add_constant(data[["Y_t_1_hat", "ν_hat"]])).fit()
# print(model4.summary())
# data['model3_pred'] = model4.fittedvalues

# Prediction exclusively of circumstances
# data['model3_pred_circum'] = model4.params['const'] + model4.params['Y_t_1_hat'] * data['Y_t_1_hat']
# Prediction exclusively of effort
# mean_circu = data['Y_t_1_hat'].mean()
# data['mean_circu'] = mean_circu
# data['model3_pred_effort'] = (model4.params['const'] + 
                          # model4.params['ν_hat'] * data['ν_hat'] + 
                          # model4.params['Y_t_1_hat'] * mean_circu)

In [18]:
# Model 3
model3 = sm.OLS(train_data["score_MAT3"], sm.add_constant(train_dummy_variables.astype(np.float64))).fit()
print(model3.summary())

# First step
train_data.loc[:, 'Y_t_1_hat'] = model3.fittedvalues
train_data.loc[:, 'ν_hat'] = model3.resid

# Second step
model4 = sm.OLS(train_data["score_MAT"], sm.add_constant(pd.concat([train_data["Y_t_1_hat"], train_data["ν_hat"]], axis = 1))).fit()
print(model4.summary())

test_data.loc[:, 'Y_t_1_hat'] = model3.predict(sm.add_constant(test_dummy_variables.astype(np.float64)))
test_data.loc[:, 'ν_hat'] = test_data["score_MAT3"] - test_data.loc[:, 'Y_t_1_hat']

test_data.loc[:, 'model3_pred'] = model4.predict(sm.add_constant(pd.concat([test_data["Y_t_1_hat"], test_data["ν_hat"]], axis = 1)))

# Prediction exclusively of circumstances
test_data.loc[:, 'model3_pred_circum'] = model4.params['const'] + model4.params['Y_t_1_hat'] * test_data['Y_t_1_hat']
# Prediction exclusively of effort
mean_circu = test_data.loc[:, 'Y_t_1_hat'].mean()
test_data.loc[:, 'mean_circu'] = mean_circu
test_data.loc[:, 'model3_pred_effort'] = (model4.params['const'] + 
                          model4.params['ν_hat'] * test_data['ν_hat'] + 
                          model4.params['Y_t_1_hat'] * mean_circu)

                            OLS Regression Results                            
Dep. Variable:             score_MAT3   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.087
Method:                 Least Squares   F-statistic:                     22.27
Date:                Wed, 18 Sep 2024   Prob (F-statistic):          3.65e-100
Time:                        15:06:37   Log-Likelihood:                -34283.
No. Observations:                5803   AIC:                         6.862e+04
Df Residuals:                    5776   BIC:                         6.880e+04
Df Model:                          26                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/1741112637.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.loc[:, 'Y_t_1_hat'] = model3.fittedvalues
/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/1741112637.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.loc[:, 'ν_hat'] = model3.resid
/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/1741112637.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

In [43]:
# Transform predictions(continuous) to quartiles(categorical)

def discretise_scores(data):
    data.loc[:, 'score_MAT_discrete'] = pd.qcut(data['score_MAT'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_discrete'] = data['score_MAT_discrete'].astype(int)

    data.loc[:, 'score_MAT_pred1'] = pd.qcut(data['model1_pred'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_pred1'] = data['score_MAT_pred1'].astype(int)
    data.loc[:, 'score_MAT_pred2'] = pd.qcut(data['model2_pred'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_pred2'] = data['score_MAT_pred2'].astype(int)
    data.loc[:, 'score_MAT_pred3'] = pd.qcut(data['model3_pred'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_pred3'] = data['score_MAT_pred3'].astype(int)
    data.loc[:, 'score_MAT_pred_C'] = pd.qcut(data['model3_pred_circum'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_pred_C'] = data['score_MAT_pred_C'].astype(int)
    data.loc[:, 'score_MAT_pred_X'] = pd.qcut(data['model3_pred_effort'], 4, labels=["1", "2", "3","4"])
    data.loc[:, 'score_MAT_pred_X'] = data['score_MAT_pred_X'].astype(int)

    # Transform predictions(continuous) to percentiles but percentiles 2 and 3 equal (between 25th and 75th percentile)

    data.loc[:, 'score_MAT_pred1_t'] = data['score_MAT_pred1'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
    data.loc[:, 'score_MAT_pred2_t'] = data['score_MAT_pred2'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
    data.loc[:, 'score_MAT_pred3_t'] = data['score_MAT_pred3'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
    data.loc[:, 'score_MAT_pred_C_t'] = data['score_MAT_pred_C'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))
    data.loc[:, 'score_MAT_pred_X_t'] = data['score_MAT_pred_X'].apply(lambda x: 1 if x == 1 else (2 if x == 2 or x == 3 else 3))

    return test_data

test_data = discretise_scores(test_data)

/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/3366699181.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[:, 'score_MAT_discrete'] = pd.qcut(data['score_MAT'], 4, labels=["1", "2", "3","4"])
/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/3366699181.py:5: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[2 4 3 ... 4 4 1]' has dtype incompatible with category, please explicitly cast to a compatible dtype first.
  data.loc[:, 'score_MAT_discrete'] = data['score_MAT_discrete'].astype(int)
/var/folders/0w/t1v535y91l76l6x3j7zj5wb00000gn/T/ipykernel_1857/3366699181.py:7: FutureWarning: Setting an item of incompatible dtype is deprecated an

## Results

We focus on **Equalized Odds** (Equality of opportunity).

To calculate Equalized Odds we first calculate recall or sensitivity:

$$TP / (TP + FN)$$

and then we calculate the ratio of recall among different groups to obtain Equalized Odds.

Recall is calculated for Low and High academic performance:
- **Low academic performance**: Below the median or 25th percentile
- **High academic performance**: Above the median or above 75th percentile (top 25 percent)

In [20]:
def compute_results(data):

    recall_dfs_25_75 = []
    recall_dfs_25_75.extend(compute_recall(data, ["f12a"], top_level=5))
    recall_dfs_25_75.extend(compute_recall(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
    recall_dfs_25_75.extend(compute_recall(data, ["start_schooling_age"], top_level=1))
    recall_dfs_25_75.extend(compute_recall(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

    recall_dfs_between25_75 = []
    recall_dfs_between25_75.extend(compute_recall_terciles(data, ["f12a"], top_level=5))
    recall_dfs_between25_75.extend(compute_recall_terciles(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
    recall_dfs_between25_75.extend(compute_recall_terciles(data, ["start_schooling_age"], top_level=1))
    recall_dfs_between25_75.extend(compute_recall_terciles(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

    recall_dfs_median = []
    recall_dfs_median.extend(compute_recall_median(data, ["f12a"], top_level=5))
    recall_dfs_median.extend(compute_recall_median(data, ["mother_education", "father_education", "mother_occupation", "father_occupation", "books"], top_level=4))
    recall_dfs_median.extend(compute_recall_median(data, ["start_schooling_age"], top_level=1))
    recall_dfs_median.extend(compute_recall_median(data, ["inmigrant_second_gen", "public_private", "capital_island", "a1", "ESCS_median", "ESCS_p25_p75", "d14"], top_level=1))

    # Combine DataFrames
    combined_df_25_75 = pd.concat(recall_dfs_25_75, ignore_index=True)
    combined_df_between25_75 = pd.concat(recall_dfs_between25_75, ignore_index=True)
    combined_df_median = pd.concat(recall_dfs_median, ignore_index=True)

    return combined_df_25_75, combined_df_between25_75, combined_df_median

combined_df_25_75, combined_df_between25_75, combined_df_median = compute_results(test_data)

In [21]:
# Pivot tables
pivot_combined_df_25_75 = combined_df_25_75.pivot_table(index=['Variable', 'Group', 'Percentile'], columns='Model', values='Recall').reset_index()
pivot_combined_df_25_75 = pivot_combined_df_25_75[['Variable', 'Group', 'Percentile', 'pred1', 'pred2', 'pred3', 'pred_C', 'pred_X']]
pivot_combined_df_25_75_sorted = pivot_combined_df_25_75.sort_values(by=['Percentile', 'Variable', 'Group'], ascending=[True, True, False])
pivot_combined_df_between25_75 = combined_df_between25_75.pivot_table(index=['Variable', 'Group', 'Tercile'], columns='Model', values='Recall').reset_index()
pivot_combined_df_between25_75 = pivot_combined_df_between25_75[['Variable', 'Group', 'Tercile', 'pred1_t', 'pred2_t', 'pred3_t', 'pred_C_t', 'pred_X_t']]
pivot_combined_df_between25_75_sorted = pivot_combined_df_between25_75.sort_values(by=['Tercile', 'Variable', 'Group'], ascending=[True, True, False])
pivot_combined_df_median = combined_df_median.pivot_table(index=['Variable', 'Group', 'Pair1', 'Pair2'], columns='Model', values='Recall').reset_index()
pivot_combined_df_median = pivot_combined_df_median[['Variable', 'Group', 'Pair1', 'Pair2', 'pred1', 'pred2', 'pred3', 'pred_C', 'pred_X']]
pivot_combined_df_median_sorted = pivot_combined_df_median.sort_values(by=['Pair1', 'Pair2', 'Variable', 'Group'], ascending=[True, True, True, False])

In [22]:
final_data_25_75 = []

for variable in pivot_combined_df_25_75_sorted['Variable'].unique():
    variable_df = pivot_combined_df_25_75_sorted[pivot_combined_df_25_75_sorted['Variable'] == variable]
    for percentile in variable_df['Percentile'].unique():
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Percentile'] == percentile)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[variable_df['Percentile'] == percentile].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Percentile': row['Percentile'],
                    'pred1': row['pred1'],
                    'pred2': row['pred2'],
                    'pred3': row['pred3'],
                    'pred_C': row['pred_C'],
                    'pred_X': row['pred_X'],
                    'pred1_odds': calculate_odds(row['pred1'], top_row['pred1']),
                    'pred2_odds': calculate_odds(row['pred2'], top_row['pred2']),
                    'pred3_odds': calculate_odds(row['pred3'], top_row['pred3']),
                    'pred_C_odds': calculate_odds(row['pred_C'], top_row['pred_C']),
                    'pred_X_odds': calculate_odds(row['pred_X'], top_row['pred_X']),
                }
                temp_data.append(odds_row)
            final_data_25_75.extend(temp_data)

final_data_25_75_sorted = pd.DataFrame(final_data_25_75)

In [23]:
final_data_between25_75 = []

for variable in pivot_combined_df_between25_75_sorted['Variable'].unique():
    variable_df = pivot_combined_df_between25_75_sorted[pivot_combined_df_between25_75_sorted['Variable'] == variable]
    for tercile in variable_df['Tercile'].unique():
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Tercile'] == tercile)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[variable_df['Tercile'] == tercile].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Tercile': row['Tercile'],
                    'pred1_t': row['pred1_t'],
                    'pred2_t': row['pred2_t'],
                    'pred3_t': row['pred3_t'],
                    'pred_C_t': row['pred_C_t'],
                    'pred_X_t': row['pred_X_t'],
                    'pred1_odds': calculate_odds(row['pred1_t'], top_row['pred1_t']),
                    'pred2_odds': calculate_odds(row['pred2_t'], top_row['pred2_t']),
                    'pred3_odds': calculate_odds(row['pred3_t'], top_row['pred3_t']),
                    'pred_C_odds': calculate_odds(row['pred_C_t'], top_row['pred_C_t']),
                    'pred_X_odds': calculate_odds(row['pred_X_t'], top_row['pred_X_t']),
                }
                temp_data.append(odds_row)
            final_data_between25_75.extend(temp_data)

final_data_between25_75_sorted = pd.DataFrame(final_data_between25_75)


In [24]:
final_data_median = []

for variable in pivot_combined_df_median_sorted['Variable'].unique():
    variable_df = pivot_combined_df_median_sorted[pivot_combined_df_median_sorted['Variable'] == variable]
    for pair in variable_df[['Pair1', 'Pair2']].drop_duplicates().values:
        pair1, pair2 = pair
        top_row = variable_df[(variable_df['Group'] == 'top') & (variable_df['Pair1'] == pair1) & (variable_df['Pair2'] == pair2)]
        if not top_row.empty:
            top_row = top_row.iloc[0]
            temp_data = []
            for _, row in variable_df[(variable_df['Pair1'] == pair1) & (variable_df['Pair2'] == pair2)].iterrows():
                odds_row = {
                    'Variable': row['Variable'],
                    'Group': row['Group'],
                    'Pair1': row['Pair1'],
                    'Pair2': row['Pair2'],
                    'pred1': row['pred1'],
                    'pred2': row['pred2'],
                    'pred3': row['pred3'],
                    'pred_C': row['pred_C'],
                    'pred_X': row['pred_X'],
                    'pred1_odds': calculate_odds(row['pred1'], top_row['pred1']),
                    'pred2_odds': calculate_odds(row['pred2'], top_row['pred2']),
                    'pred3_odds': calculate_odds(row['pred3'], top_row['pred3']),
                    'pred_C_odds': calculate_odds(row['pred_C'], top_row['pred_C']),
                    'pred_X_odds': calculate_odds(row['pred_X'], top_row['pred_X']),
                }
                temp_data.append(odds_row)
            final_data_median.extend(temp_data)

final_data_median_sorted = pd.DataFrame(final_data_median)


In [25]:
category_order = ['a1', 'mother_education', 'father_education', 'mother_occupation', 'father_occupation', 'books', 'd14', 'inmigrant_second_gen', 
                  'public_private', 'capital_island', 'start_schooling_age', 'f12a', 'ESCS_median', 'ESCS_p25_p75']

final_data_25_75_sorted['Variable'] = pd.Categorical(final_data_25_75_sorted['Variable'], categories=category_order, ordered=True)
final_data_25_75_sorted = final_data_25_75_sorted.sort_values(by='Variable')
final_data_25_75_sorted = final_data_25_75_sorted[['Variable', 'Group', 'Percentile', 'pred1', 'pred1_odds', 'pred2', 'pred2_odds', 'pred3', 'pred3_odds', 'pred_C', 'pred_C_odds', 'pred_X', 'pred_X_odds']]
final_data_25_75_sorted = final_data_25_75_sorted.sort_values(by=['Percentile', 'Variable', 'Group'], ascending=[True, True, False])
final_data_between25_75_sorted['Variable'] = pd.Categorical(final_data_between25_75_sorted['Variable'], categories=category_order, ordered=True)
final_data_between25_75_sorted = final_data_between25_75_sorted.sort_values(by='Variable')
final_data_between25_75_sorted = final_data_between25_75_sorted[['Variable', 'Group', 'Tercile', 'pred1_t', 'pred1_odds', 'pred2_t', 'pred2_odds', 'pred3_t', 'pred3_odds', 'pred_C_t', 'pred_C_odds', 'pred_X_t', 'pred_X_odds']]
final_data_between25_75_sorted = final_data_between25_75_sorted.sort_values(by=['Tercile', 'Variable', 'Group'], ascending=[True, True, False])
final_data_median_sorted['Variable'] = pd.Categorical(final_data_median_sorted['Variable'], categories=category_order, ordered=True)
final_data_median_sorted = final_data_median_sorted.sort_values(by='Variable')
final_data_median_sorted = final_data_median_sorted[['Variable', 'Group', 'Pair1', 'Pair2', 'pred1', 'pred1_odds', 'pred2', 'pred2_odds', 'pred3', 'pred3_odds', 'pred_C', 'pred_C_odds', 'pred_X', 'pred_X_odds']]
final_data_median_sorted = final_data_median_sorted.sort_values(by=['Pair1', 'Pair2', 'Variable', 'Group'], ascending=[True, True, True, False])

## Export to Excel

In [35]:
# Export to Excel
with pd.ExcelWriter(os.path.join('results', 'results.xlsx')) as writer:
    final_data_25_75_sorted.to_excel(writer, sheet_name='25_75', index=False, float_format='%.4f')
    final_data_median_sorted.to_excel(writer, sheet_name='Median', index=False, float_format='%.4f')
    final_data_between25_75_sorted.to_excel(writer, sheet_name='between25_75', index=False, float_format='%.4f')
    data.to_excel(writer, sheet_name='data', index=False, float_format='%.4f')

## IOP

Inequality of Opportunity is computed by applying an inequality index (Gini, MLD or simple variance) to the set of central moments (specifically, the mean $\mu$) for the $Y$'s conditional distributions with respect to a sensitive attribute's values. Mathematically:
$$IOP = I(w_{i} * \mu(Y|G_{i}), \ldots, w_{m} * \mu(Y|G_{m}))$$ 

$I$ is the inequality index while $G_{1} \ldots G_{m}$ are the $m$ different groups of individuals identified by the values a given senstive attribute can have. For example, if _gender_ is a sensitive attribute then the two resulting groups might be $G_{1} = male$ and $G_{2} = female$. In this situation, _IOP_ would be absent if $$I(w_{1} * \mu(Y|G_{1}), w_{2} * \mu(Y|G_{2})) = 0$$ or close to 0.

Finally, each central moment is weighted by the fraction of samples with a given value of the sensitive attribute. The weights sum up to 1. In the previous example, $w_{1}$ is the fraction of elements with $G = m$ and $w_{2}$ is the fraction of elements with $G = f$. 

In [25]:
test_data.columns

Index(['id_student_16_19', 'score_MAT', 'score_LEN', 'score_MAT3',
       'score_LEN3', 'a1', 'mother_education', 'father_education',
       'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
       'start_schooling_age', 'books', 'f12a', 'public_private',
       'capital_island', 'd14', 'ESCS', 'id_school', 'score_MATq',
       'score_LENq', 'ESCS_median', 'ESCS_p25_p75', 'model1_pred',
       'model2_pred', 'Y_t_1_hat', 'ν_hat', 'model3_pred',
       'model3_pred_circum', 'mean_circu', 'model3_pred_effort',
       'score_MAT_pred1', 'score_MAT_pred2', 'score_MAT_pred3',
       'score_MAT_pred_C', 'score_MAT_pred_X', 'score_MAT_pred1_t',
       'score_MAT_pred2_t', 'score_MAT_pred3_t', 'score_MAT_pred_C_t',
       'score_MAT_pred_X_t'],
      dtype='object')

## Utility

In [54]:
def compute_accuracy(test_data, model_pred, sa, protected_group):
    # filter gts and preds based on values of the sensitive attributes
    # protected group (normally one value) vs non protected group(s) (potentially multiple values)
    # compute accuracy for each group
    if test_data[sa].isna().sum() > 0:
        test_data = test_data.dropna(axis=0)
    else:
        test_data = test_data.dropna(axis=1)

    y_true_protected = test_data.loc[test_data[sa] == protected_group]["score_MAT_discrete"].astype(np.int64)
    y_pred_protected = test_data.loc[test_data[sa] == protected_group][model_pred].astype(np.int64)

    y_true_non_protected = test_data.loc[~(test_data[sa].astype(np.int64) == protected_group)]["score_MAT_discrete"]
    y_pred_non_protected = test_data.loc[~(test_data[sa].astype(np.int64) == protected_group)][model_pred]

    accuracy_protected = round(accuracy_score(y_true_protected, y_pred_protected), 2)
    accuracy_non_protected = round(accuracy_score(y_true_non_protected, y_pred_non_protected), 2)

    return accuracy_protected, accuracy_non_protected

def run_experiments(test_data, percentile, ineq_index):
    sensitive_attrs = ['a1', 'mother_education', 'father_education',
       'mother_occupation', 'father_occupation', 'inmigrant_second_gen',
       'start_schooling_age', 'books', 'f12a', 'public_private',
       'capital_island', 'd14', "ESCS_median", "ESCS_p25_p75"]

    columns = ["Model 1", "Model 2", "Model 3", "Circumstances", "Effort"]
    columns_acc = columns + ["Model 1 terciles", "Model 2 terciles", "Model 3 terciles", "Circumstances terciles", "Effort terciles"]
    model_preds_acc = [f"score_MAT_{x}" for x in ["pred1", "pred2", "pred3", "pred_C", "pred_X", "pred1_t", "pred2_t", "pred3_t", "pred_C_t", "pred_X_t"]]
    model_preds = ["model1_pred", "model2_pred", "model3_pred", "model3_pred_circum", "model3_pred_effort"]

    model_pred_rename = {mp: col for mp, col in zip(model_preds, columns)}
    model_pred_acc_rename = {mpa: col for mpa, col in zip(model_preds_acc, columns_acc)}

    sensitive_attrs_values = {
         "a1": 1,
         "mother_education": 4,
         "father_education": 4,
         "mother_occupation": 4,
         "father_occupation": 4,
         "inmigrant_second_gen": 1, 
         "start_schooling_age": 1, 
         "books": 4, 
         "f12a": 5,
         "public_private": 1,
         "capital_island": 1,
         "d14": 1,
         "ESCS_median": 1,
         "ESCS_p25_p75": 1
    }
    
    df = {}
    for mp in model_preds:
        col = []
        test_data.loc[:, "label"] = binarise_predictions(test_data[mp], percentile)
        for sa in sensitive_attrs:
            col.append(iop(test_data, sa, ineq_index=ineq_index))
        df[model_pred_rename[mp]] = col

    df_accuracy = {}
    for mp in model_preds_acc:
        col_acc = []
        for sa in sensitive_attrs:
            col_acc.append(compute_accuracy(test_data, mp, sa, sensitive_attrs_values[sa]))
        df_accuracy[model_pred_acc_rename[mp]] = col_acc
    return pd.DataFrame(df, index=sensitive_attrs), pd.DataFrame(df_accuracy, index=sensitive_attrs)

## Below 25th percentile

In [55]:
df_below_25, df_acc_below_25 = run_experiments(test_data, percentile="below-25", ineq_index="gini")
df_below_25

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.027127,0.008253,0.007857,0.030885,0.035137
mother_education,0.381762,0.284920,0.254914,0.340101,0.504174
father_education,0.166426,0.263529,0.220694,0.370028,0.265515
mother_occupation,0.630236,0.622347,0.624253,0.615724,0.633293
father_occupation,0.619720,0.614145,0.613615,0.598024,0.624733
inmigrant_second_gen,0.494131,0.494961,0.493568,0.494824,0.494409
start_schooling_age,0.533531,0.504562,0.504593,0.423612,0.556374
books,0.397864,0.455104,0.459491,0.548036,0.364081
f12a,0.571787,0.547349,0.552203,0.543713,0.589526
public_private,0.438688,0.446869,0.456131,0.475014,0.404293


In [56]:
df_acc_below_25

,Model 1,Model 2,Model 3,Circumstances,Effort,Model 1 terciles,Model 2 terciles,Model 3 terciles,Circumstances terciles,Effort terciles
a1,"(0.44, 0.4)","(0.43, 0.4)","(0.44, 0.41)","(0.32, 0.32)","(0.41, 0.38)","(0.36, 0.32)","(0.36, 0.34)","(0.36, 0.34)","(0.29, 0.29)","(0.34, 0.3)"
mother_education,"(0.43, 0.42)","(0.42, 0.41)","(0.43, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.3, 0.38)","(0.31, 0.38)","(0.32, 0.38)","(0.28, 0.3)","(0.29, 0.35)"
father_education,"(0.43, 0.42)","(0.42, 0.42)","(0.42, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.28, 0.38)","(0.3, 0.38)","(0.3, 0.38)","(0.25, 0.31)","(0.27, 0.35)"
mother_occupation,"(0.44, 0.42)","(0.44, 0.41)","(0.46, 0.41)","(0.34, 0.32)","(0.42, 0.39)","(0.29, 0.36)","(0.32, 0.36)","(0.33, 0.36)","(0.29, 0.29)","(0.27, 0.34)"
father_occupation,"(0.43, 0.42)","(0.42, 0.42)","(0.44, 0.42)","(0.35, 0.31)","(0.4, 0.39)","(0.31, 0.36)","(0.33, 0.36)","(0.33, 0.36)","(0.28, 0.29)","(0.29, 0.34)"
inmigrant_second_gen,"(0.42, 0.43)","(0.42, 0.41)","(0.43, 0.39)","(0.33, 0.29)","(0.39, 0.43)","(0.34, 0.38)","(0.35, 0.36)","(0.35, 0.38)","(0.29, 0.23)","(0.32, 0.35)"
start_schooling_age,"(0.43, 0.41)","(0.42, 0.41)","(0.43, 0.41)","(0.32, 0.33)","(0.4, 0.39)","(0.33, 0.38)","(0.33, 0.39)","(0.33, 0.39)","(0.27, 0.32)","(0.32, 0.34)"
books,"(0.45, 0.42)","(0.42, 0.41)","(0.44, 0.42)","(0.36, 0.31)","(0.4, 0.39)","(0.31, 0.35)","(0.33, 0.36)","(0.34, 0.36)","(0.29, 0.29)","(0.28, 0.33)"
f12a,"(0.44, 0.42)","(0.44, 0.4)","(0.45, 0.41)","(0.33, 0.31)","(0.41, 0.39)","(0.34, 0.35)","(0.35, 0.35)","(0.36, 0.35)","(0.28, 0.29)","(0.31, 0.33)"
public_private,"(0.42, 0.42)","(0.4, 0.42)","(0.43, 0.42)","(0.33, 0.32)","(0.39, 0.4)","(0.29, 0.36)","(0.29, 0.37)","(0.31, 0.37)","(0.27, 0.3)","(0.29, 0.33)"


## Between 25th and 75th percentile

In [60]:
df_between_25_75, df_acc_between_25_75 = run_experiments(test_data, percentile="between-25-75", ineq_index="gini")
df_between_25_75

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.051123,0.053941,0.049149,0.039552,0.062716
mother_education,0.497542,0.498034,0.502385,0.506509,0.492356
father_education,0.271492,0.254522,0.261380,0.216622,0.264808
mother_occupation,0.642399,0.644819,0.644120,0.666278,0.634566
father_occupation,0.622298,0.632355,0.629451,0.644177,0.611308
inmigrant_second_gen,0.494340,0.494056,0.494613,0.492632,0.494335
start_schooling_age,0.544887,0.548175,0.547645,0.560247,0.549210
books,0.382921,0.394541,0.387748,0.479159,0.369345
f12a,0.571758,0.579884,0.577035,0.567326,0.570528
public_private,0.384719,0.389518,0.386662,0.404977,0.391946


In [61]:
df_acc_between_25_75

,Model 1,Model 2,Model 3,Circumstances,Effort,Model 1 terciles,Model 2 terciles,Model 3 terciles,Circumstances terciles,Effort terciles
a1,"(0.44, 0.4)","(0.43, 0.4)","(0.44, 0.41)","(0.32, 0.32)","(0.41, 0.38)","(0.36, 0.32)","(0.36, 0.34)","(0.36, 0.34)","(0.29, 0.29)","(0.34, 0.3)"
mother_education,"(0.43, 0.42)","(0.42, 0.41)","(0.43, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.3, 0.38)","(0.31, 0.38)","(0.32, 0.38)","(0.28, 0.3)","(0.29, 0.35)"
father_education,"(0.43, 0.42)","(0.42, 0.42)","(0.42, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.28, 0.38)","(0.3, 0.38)","(0.3, 0.38)","(0.25, 0.31)","(0.27, 0.35)"
mother_occupation,"(0.44, 0.42)","(0.44, 0.41)","(0.46, 0.41)","(0.34, 0.32)","(0.42, 0.39)","(0.29, 0.36)","(0.32, 0.36)","(0.33, 0.36)","(0.29, 0.29)","(0.27, 0.34)"
father_occupation,"(0.43, 0.42)","(0.42, 0.42)","(0.44, 0.42)","(0.35, 0.31)","(0.4, 0.39)","(0.31, 0.36)","(0.33, 0.36)","(0.33, 0.36)","(0.28, 0.29)","(0.29, 0.34)"
inmigrant_second_gen,"(0.42, 0.43)","(0.42, 0.41)","(0.43, 0.39)","(0.33, 0.29)","(0.39, 0.43)","(0.34, 0.38)","(0.35, 0.36)","(0.35, 0.38)","(0.29, 0.23)","(0.32, 0.35)"
start_schooling_age,"(0.43, 0.41)","(0.42, 0.41)","(0.43, 0.41)","(0.32, 0.33)","(0.4, 0.39)","(0.33, 0.38)","(0.33, 0.39)","(0.33, 0.39)","(0.27, 0.32)","(0.32, 0.34)"
books,"(0.45, 0.42)","(0.42, 0.41)","(0.44, 0.42)","(0.36, 0.31)","(0.4, 0.39)","(0.31, 0.35)","(0.33, 0.36)","(0.34, 0.36)","(0.29, 0.29)","(0.28, 0.33)"
f12a,"(0.44, 0.42)","(0.44, 0.4)","(0.45, 0.41)","(0.33, 0.31)","(0.41, 0.39)","(0.34, 0.35)","(0.35, 0.35)","(0.36, 0.35)","(0.28, 0.29)","(0.31, 0.33)"
public_private,"(0.42, 0.42)","(0.4, 0.42)","(0.43, 0.42)","(0.33, 0.32)","(0.39, 0.4)","(0.29, 0.36)","(0.29, 0.37)","(0.31, 0.37)","(0.27, 0.3)","(0.29, 0.33)"


## Above 75th percentile

In [62]:
df_above_75, df_acc_above_75 = run_experiments(test_data, percentile="above-75", ineq_index="gini")
df_above_75

,Model 1,Model 2,Model 3,Circumstances,Effort
a1,0.048770,0.078193,0.071835,0.128794,0.017500
mother_education,0.577700,0.636254,0.636273,0.729445,0.489083
father_education,0.388212,0.506141,0.478017,0.662430,0.267675
mother_occupation,0.622400,0.624908,0.624560,0.604339,0.635541
father_occupation,0.591286,0.574718,0.581993,0.563150,0.608774
inmigrant_second_gen,0.494400,0.494131,0.494409,0.496971,0.494131
start_schooling_age,0.571968,0.589380,0.590437,0.621248,0.541746
books,0.324200,0.298617,0.309552,0.474779,0.366200
f12a,0.605839,0.612703,0.613851,0.650964,0.590882
public_private,0.360945,0.339275,0.332597,0.250756,0.385952


In [63]:
df_acc_above_75

,Model 1,Model 2,Model 3,Circumstances,Effort,Model 1 terciles,Model 2 terciles,Model 3 terciles,Circumstances terciles,Effort terciles
a1,"(0.44, 0.4)","(0.43, 0.4)","(0.44, 0.41)","(0.32, 0.32)","(0.41, 0.38)","(0.36, 0.32)","(0.36, 0.34)","(0.36, 0.34)","(0.29, 0.29)","(0.34, 0.3)"
mother_education,"(0.43, 0.42)","(0.42, 0.41)","(0.43, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.3, 0.38)","(0.31, 0.38)","(0.32, 0.38)","(0.28, 0.3)","(0.29, 0.35)"
father_education,"(0.43, 0.42)","(0.42, 0.42)","(0.42, 0.42)","(0.34, 0.31)","(0.4, 0.4)","(0.28, 0.38)","(0.3, 0.38)","(0.3, 0.38)","(0.25, 0.31)","(0.27, 0.35)"
mother_occupation,"(0.44, 0.42)","(0.44, 0.41)","(0.46, 0.41)","(0.34, 0.32)","(0.42, 0.39)","(0.29, 0.36)","(0.32, 0.36)","(0.33, 0.36)","(0.29, 0.29)","(0.27, 0.34)"
father_occupation,"(0.43, 0.42)","(0.42, 0.42)","(0.44, 0.42)","(0.35, 0.31)","(0.4, 0.39)","(0.31, 0.36)","(0.33, 0.36)","(0.33, 0.36)","(0.28, 0.29)","(0.29, 0.34)"
inmigrant_second_gen,"(0.42, 0.43)","(0.42, 0.41)","(0.43, 0.39)","(0.33, 0.29)","(0.39, 0.43)","(0.34, 0.38)","(0.35, 0.36)","(0.35, 0.38)","(0.29, 0.23)","(0.32, 0.35)"
start_schooling_age,"(0.43, 0.41)","(0.42, 0.41)","(0.43, 0.41)","(0.32, 0.33)","(0.4, 0.39)","(0.33, 0.38)","(0.33, 0.39)","(0.33, 0.39)","(0.27, 0.32)","(0.32, 0.34)"
books,"(0.45, 0.42)","(0.42, 0.41)","(0.44, 0.42)","(0.36, 0.31)","(0.4, 0.39)","(0.31, 0.35)","(0.33, 0.36)","(0.34, 0.36)","(0.29, 0.29)","(0.28, 0.33)"
f12a,"(0.44, 0.42)","(0.44, 0.4)","(0.45, 0.41)","(0.33, 0.31)","(0.41, 0.39)","(0.34, 0.35)","(0.35, 0.35)","(0.36, 0.35)","(0.28, 0.29)","(0.31, 0.33)"
public_private,"(0.42, 0.42)","(0.4, 0.42)","(0.43, 0.42)","(0.33, 0.32)","(0.39, 0.4)","(0.29, 0.36)","(0.29, 0.37)","(0.31, 0.37)","(0.27, 0.3)","(0.29, 0.33)"


In [ ]:
np_means = np.concatenate([
    df_below_25.mean(axis=0).to_numpy().reshape((1, df_below_25.shape[1])),
    df_above_75.mean(axis=0).to_numpy().reshape((1, df_above_75.shape[1])),
    df_between_25_75.mean(axis=0).to_numpy().reshape((1, df_between_25_75.shape[1]))], axis=0)

In [ ]:
df_means = pd.DataFrame(np_means, 
                        index=["below_25", "above_75", "between_25_75"], 
                        columns=["Model 1", "Model 2", "Model 3", "Circumstances", "Effort"])
df_means

,Model 1,Model 2,Model 3,Circumstances,Effort
below_25,0.383389,0.402007,0.398640,0.443226,0.375765
above_75,0.398160,0.428580,0.422254,0.472565,0.369460
between_25_75,0.373163,0.376371,0.374766,0.384849,0.373052
